In [2]:
print('lets get these things imported')
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt


import category_encoders as ce
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import plot_tree


lets get these things imported


In [3]:
dfreal = pd.read_csv('True.csv',
                    parse_dates = ['date'])
                    #index_col = 'date')
dfreal['Fake'] = 0
dfreal.shape


(21417, 5)

In [4]:
dfreal.head()

,title,text,subject,date,Fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,2017-12-31,0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,2017-12-29,0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,2017-12-31,0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,2017-12-30,0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,2017-12-29,0


In [5]:
dfreal_trimmed = dfreal[-21_400 :]
dfreal_trimmed.shape

(21400, 5)

In [6]:

dffake = pd.read_csv('Fake.csv',
                    parse_dates = ['date'])
                    #index_col = 'date')

dffake['Fake'] = 1
dffake.shape

(23481, 5)

In [7]:
dffake.head()

,title,text,subject,date,Fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [8]:
# Alright, we should be happy with that. Shame it took so long to get there. 
# We are adding a boolean column for "Fake" (0 will mean real news)


In [9]:
dffake_trimmed = dffake[-21_400 :]
print(dffake_trimmed.shape)

(21400, 5)


In [10]:
# frames = [dfreal_trimmed, dffake_trimmed]
# df_joined = pd.concat(frames)
# df_joined.shape
df_joined = dfreal_trimmed.append(dffake_trimmed, ignore_index=True)
# Trimming everything up to be right around (42_000, 5) or so merged length with  
df_joined.shape

(42800, 5)

In [11]:
df_joined.head()

,title,text,subject,date,Fake
0,"Trump on Twitter (Dec 26) - Hillary Clinton, T...",The following statements were posted to the ve...,politicsNews,2017-12-26 00:00:00,0
1,U.S. appeals court rejects challenge to Trump ...,(Reuters) - A U.S. appeals court in Washington...,politicsNews,2017-12-26 00:00:00,0
2,Treasury Secretary Mnuchin was sent gift-wrapp...,(Reuters) - A gift-wrapped package addressed t...,politicsNews,2017-12-24 00:00:00,0
3,Federal judge partially lifts Trump's latest r...,WASHINGTON (Reuters) - A federal judge in Seat...,politicsNews,2017-12-24 00:00:00,0
4,Exclusive: U.S. memo weakens guidelines for pr...,NEW YORK (Reuters) - The U.S. Justice Departme...,politicsNews,2017-12-23 00:00:00,0


In [12]:
df_joined.describe()

,Fake
count,42800.000000
mean,0.500000
std,0.500006
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [13]:
# Now I need to pass this through the NLP functions. Let's get those set up... 

# first a single row just to test... 

df_test01 = df_joined[: 2]
df_test01.head()

,title,text,subject,date,Fake
0,"Trump on Twitter (Dec 26) - Hillary Clinton, T...",The following statements were posted to the ve...,politicsNews,2017-12-26 00:00:00,0
1,U.S. appeals court rejects challenge to Trump ...,(Reuters) - A U.S. appeals court in Washington...,politicsNews,2017-12-26 00:00:00,0


# Count Vectorizer

In [14]:
from pandas import DataFrame
cv = CountVectorizer(stop_words = 'english', max_features = 1000, max_df = 0.5 , min_df = 0.05 )
# test01_trans = cv.fit_transform(df_test01).toarray()
# dftest_cv = pd.DataFrame(test01_trans,
#                         columns=cv.get_feature_names())

#change index labels - ask why
#dftest_cv.index = [f'doc_{n}' for n in dftest_cv.index]

def create_term_matrix(message_list, vectorizer):
    doc_term_df = vectorizer.fit_transform(message_list)
    return DataFrame(doc_term_df.toarray(),
                     columns=vectorizer.get_feature_names())
                    
#create_term_matrix(df_test01['text'], cv)

#df_joined_countvectored = 

create_term_matrix(df_joined['text'], cv)

#df_joined_countvectored.shape()

,000,10,100,11,12,15,20,2012,2014,2015,...,words,work,worked,working,world,wrote,year,years,york,young
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,4,0,0,0,0,0,1,...,0,0,0,0,0,0,1,1,0,0
4,0,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42795,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,1,0
42796,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42797,2,0,2,0,0,1,1,0,1,4,...,2,5,1,0,4,0,1,6,2,0
42798,1,0,0,0,0,0,0,2,0,1,...,0,1,1,0,0,0,1,0,0,0


# next we try for - TFIDF Vectorizer -

In [15]:
# # Instantiate vectorizer
# tfidf = TfidfVectorizer(stop_words = 'english')

# # Vectorize bad_revs
# test_02_trans = tfidf.fit_transform(df_test01['text']).toarray()

# # Create DataFrame for display below
# dftest_tfidf = pd.DataFrame(np.round(test_02_trans, 2),
#                         columns=tfidf.get_feature_names())

# # Change index labels
# dftest_tfidf.index = [f'doc_{n}' for n in dftest_tfidf.index]

# # Show result
# dftest_tfidf

# and now we can try out Hashing Vectorizer

In [16]:
# # Instantiate vectorizer
# hv = HashingVectorizer(alternate_sign=False, norm=None, stop_words = 'english')

# # Vectorize bad_revs
# test_03_trans_hv = hv.transform(df_test01).toarray()

# # Create DataFrame for display below
# df_test_hv = pd.DataFrame(test_03_trans_hv)

# # Change index labels
# df_test_hv.index = [f'doc_{n}' for n in df_test_hv.index]

# # Show result
# df_test_hv

# I'm ready to to build a pipeline out for a random forest model but these results don't really seem actionable. I spent too much of today on the VSCode setup, but generally speaking it was all worth it. 

# obviously the target here will be "Fake" - the question is how to split it. 

In [17]:
# echo "# News_Parsing_DS23_unit02" >> README.md
# git init
# git add README.md
# git commit -m "first commit"
# git branch -M main
# git remote add origin https://github.com/StephenSpicer/News_Parsing_DS23_unit02.git
# git push -u origin main